In [1]:
import math
import datetime
import pandas as pd
from scipy import optimize
from dateutil.relativedelta import relativedelta

In [2]:
def date(num):
    num = str(num)
    day = num[-2:]
    mon = num[-4:-2]
    year = num[-6:-4]
    return f"{year}/{mon}/{day}"

def percent(num):
    return num*0.01

# Calc Price, Duration, M_Duration, Conveixty

In [3]:
class calc_bond:
    
    @staticmethod
    def date(num):
        date = datetime.datetime.strptime(num, "%y/%m/%d").date()
        return date
    
    @staticmethod
    def truncate(number, digits) -> float:
        nbDecimals = len(str(number).split('.')[1]) 
        if nbDecimals <= digits:
            return number
        stepper = 10.0 ** digits
        return math.trunc(stepper * number) / stepper
    
    # get required date, df
    def calc_df(self, issuedate, maturity, settlement, CR, YTM, period):
        
        # init
        K = int(12/period)
        schedule = self.date(issuedate)
        interval = relativedelta(months=period)
        
        i = 0
        after_cd = self.date(issuedate) + interval
        before_cd = self.date(issuedate)
        
        while schedule < self.date(maturity):
            
            # coupon num, schedule date
            i += 1
            schedule += interval
            
            # 직전이자 직후이자 구하기
            if self.date(settlement) >= schedule:
                before_cd = schedule   
                after_cd = schedule + interval 
            
            # DF of comp and simp(after_cd, before_cd 계산 이후 init)
            DF_comp_init = 1/(1+YTM/K)
            DF_simp_init = 1/(1+YTM/K*((after_cd - self.date(settlement))/(after_cd - before_cd)))    
            
        return after_cd, before_cd, DF_comp_init, DF_simp_init


    def get_price(self, issuedate, maturity, settlement, CR, YTM, period):

        after_cd, before_cd, DF_comp_init, DF_simp_init \
        = self.calc_df(issuedate, maturity, settlement, CR, YTM, period)
       
        K = int(12/period)
        interval = relativedelta(months=period)
        schedule = after_cd - interval #  첫 스케줄 init
        CF = 10000*CR/K
        PV_CF = 0
        Price = 0
        i = 0
        df_CF = pd.DataFrame(columns = ['num','date','CF', 'DF', 'PV of CF'])
        
        
        while schedule < self.date(maturity):
            
            i += 1
            schedule += interval

            # 만기 + Principle ~ 규칙인 case만 cover中
            if schedule == self.date(maturity): 
                CF += 10000

            DF_comp = DF_comp_init ** (i-1)
            DF_simp = DF_simp_init
#             DF = round(DF_comp*DF_simp, 4) # discount factor 소수점 4자리 까지
            DF = DF_comp*DF_simp

            PV_CF = CF*DF

            df_CF.loc[i] = [i, schedule, CF, DF, PV_CF] # dataframe CF 정리
            Price += PV_CF 

        df_CF.set_index(keys='num')
        print(f"[Price]: {Price}")
        print(f"YTM: {YTM}")
        print(f"[Cash Flow]\n{df_CF}\n")
        return Price
    
    
    def calc_dur_conv(self, issuedate, maturity, settlement, CR, YTM, period):
        
        # init
        K = int(12/period)
        CF = 10000*CR/K
        PV_CF = 0
        interval = relativedelta(months=period)
        
        after_cd, before_cd, DF_comp_init, DF_simp_init\
        = self.calc_df(issuedate, maturity, settlement, CR, YTM, period)
        schedule = after_cd - interval #  첫 스케줄 init
        dirty_period = (after_cd - self.date(settlement))/(after_cd - before_cd)
        price = self.get_price(issuedate, maturity, settlement, CR, YTM, period)
        
        i = 0
        tk = 0 # df of dur
        ttkk = 0 # df of conv
        duration = 0
        convexity = 0
        
        df_CF = pd.DataFrame(columns = ['num','date','CF', 'DF', 'W', 'tk',' ttkk'])
        
        while schedule < self.date(maturity):
            i += 1
            schedule += interval

            # 만기 + Principle
            if schedule == self.date(maturity): 
                CF += 10000

            DF_comp = DF_comp_init ** (i-1)
            DF_simp = DF_simp_init
#             DF = round(DF_comp*DF_simp, 4) # 소수점 5자리 절사
            DF = DF_comp*DF_simp
            PV_CF = CF*DF
            weight = PV_CF/price
            
            if i == 1: # diry_day 기준
                tk = i/K*dirty_period             
                ttkk = dirty_period*(dirty_period+1)/(K**2)
        
            else:
                tk = (dirty_period + (i-1))/K 
                ttkk = ((dirty_period+i-1)*(dirty_period+i))/(K**2)   
        
            duration += weight*tk
            convexity += weight * ttkk * (1/(1+YTM/K)**2)
            df_CF.loc[i] = [i, schedule, CF,DF, weight, tk, ttkk] # dataframe CF 정리
        M_duration = duration/(1+YTM/K)
        
        
        print(f"[Duration]: {duration}, [M_Duration]: {M_duration},[Convexity]: {convexity}\n")
        print(f"[Cash Flow]\n {df_CF}")
        
        return duration, M_duration, convexity

### Ex1

In [4]:
calc1 = calc_bond()

# 입력값
issuedate = "14/03/10"
maturity = "19/03/10"
# settlement = "14/03/10"     # 기간
settlement = "15/02/23"     # 기간


CR = 0.03125
YTM = 0.022   # 금리
# YTM = 0.03125   # 금리

period = 6   # 주기

# calc1.calc_df(issuedate, maturity, settlement, CR, YTM, period)
# calc1.get_price(issuedate, maturity, settlement, CR, YTM, period)
dur, m_dur, conv = calc1.calc_dur_conv(issuedate, maturity, settlement, CR, YTM, period)

[Price]: 10499.015833370093
YTM: 0.022
[Cash Flow]
   num        date        CF        DF     PV of CF
1    1  2015-03-10    156.25  0.999089   156.107692
2    2  2015-09-10    156.25  0.988219   154.409191
3    3  2016-03-10    156.25  0.977467   152.729170
4    4  2016-09-10    156.25  0.966832   151.067428
5    5  2017-03-10    156.25  0.956312   149.423767
6    6  2017-09-10    156.25  0.945907   147.797989
7    7  2018-03-10    156.25  0.935615   146.189900
8    8  2018-09-10    156.25  0.925436   144.599308
9    9  2019-03-10  10156.25  0.915367  9296.691389

[Duration]: 3.7804831395389633, [M_Duration]: 3.7393502863886883,[Convexity]: 16.494213300644567

[Cash Flow]
    num        date        CF        DF         W        tk       ttkk
1    1  2015-03-10    156.25  0.999089  0.014869  0.041436   0.022435
2    2  2015-09-10    156.25  0.988219  0.014707  0.541436   0.563872
3    3  2016-03-10    156.25  0.977467  0.014547  1.041436   1.605308
4    4  2016-09-10    156.25  0.96683

### Ex2

In [5]:
calc2 = calc_bond()

# 입력값
issuedate = "22/12/19"
maturity = "24/12/19"
# settlement = "14/03/10"     # 기간
settlement = "22/12/19"     # 기간


# CR = 0.03125
CR = 0.042
# YTM = 0.022   # 금리
YTM = 0.04085   # 금리

period = 6   # 주기

# calc1.calc_df(issuedate, maturity, settlement, CR, YTM, period)
# calc2.get_price(issuedate, maturity, settlement, CR, YTM, period)
dur, m_dur, conv = calc1.calc_dur_conv(issuedate, maturity, settlement, CR, YTM, period)
print(dur, m_dur, conv)

[Price]: 10021.871877810438
YTM: 0.04085
[Cash Flow]
   num        date       CF        DF     PV of CF
1    1  2023-06-19    210.0  0.979984   205.796604
2    2  2023-12-19    210.0  0.960368   201.677345
3    3  2024-06-19    210.0  0.941145   197.640537
4    4  2024-12-19  10210.0  0.922307  9416.757392

[Duration]: 1.9392136991102835, [M_Duration]: 1.9003980685599469,[Convexity]: 4.607577062253325

[Cash Flow]
    num        date       CF        DF         W   tk   ttkk
1    1  2023-06-19    210.0  0.979984  0.020535  0.5    0.5
2    2  2023-12-19    210.0  0.960368  0.020124  1.0    1.5
3    3  2024-06-19    210.0  0.941145  0.019721  1.5    3.0
4    4  2024-12-19  10210.0  0.922307  0.939621  2.0    5.0
1.9392136991102835 1.9003980685599469 4.607577062253325


### EX3  KR103501G9C9

In [6]:
calc = calc_bond()

# 입력값
issuedate = date(191210)
maturity = date(20221210)
settlement = date(20181010)     # 기간


CR = 0.0125
YTM = 0.00882   # 금리

period = 6   # 주기

dur, m_dur, conv = calc.calc_dur_conv(issuedate, maturity, settlement, CR, YTM, period)
print(dur, m_dur, conv)

[Price]: 10006.441700819065
YTM: 0.00882
[Cash Flow]
   num        date       CF        DF     PV of CF
1    1  2020-06-10     62.5  0.985536    61.596023
2    2  2020-12-10     62.5  0.981209    61.325577
3    3  2021-06-10     62.5  0.976901    61.056319
4    4  2021-12-10     62.5  0.972612    60.788243
5    5  2022-06-10     62.5  0.968341    60.521343
6    6  2022-12-10  10062.5  0.964090  9701.154196

[Duration]: 4.118036531944722, [M_Duration]: 4.099955727187823,[Convexity]: 18.93209026752629

[Cash Flow]
    num        date       CF        DF         W        tk       ttkk
1    1  2020-06-10     62.5  0.985536  0.006156  1.663934   3.600645
2    2  2020-12-10     62.5  0.981209  0.006129  2.163934   5.764579
3    3  2021-06-10     62.5  0.976901  0.006102  2.663934   8.428514
4    4  2021-12-10     62.5  0.972612  0.006075  3.163934  11.592448
5    5  2022-06-10     62.5  0.968341  0.006048  3.663934  15.256383
6    6  2022-12-10  10062.5  0.964090  0.969491  4.163934  19.42031

### KR103501GA68

In [7]:
calc = calc_bond()

# 입력값
issuedate = date(200610)
maturity = date(230610)
settlement = date(210312)

CR = percent(1)
YTM = percent(0.97)   # 금리
period = 6   # 주기

dur, m_dur, conv = calc.calc_dur_conv(issuedate, maturity, settlement, CR, YTM, period)
# print(dur, m_dur, conv)

[Price]: 10031.868002571711
YTM: 0.0097
[Cash Flow]
   num        date       CF        DF     PV of CF
1    1  2021-06-10     50.0  0.997607    49.880369
2    2  2021-12-10     50.0  0.992792    49.639617
3    3  2022-06-10     50.0  0.988001    49.400027
4    4  2022-12-10     50.0  0.983232    49.161593
5    5  2023-06-10  10050.0  0.978486  9833.786396

[Duration]: 2.222511493867485, [M_Duration]: 2.2117843398193613,[Convexity]: 6.028751898487068

[Cash Flow]
    num        date       CF        DF         W        tk      ttkk
1    1  2021-06-10     50.0  0.997607  0.004972  0.247253  0.184760
2    2  2021-12-10     50.0  0.992792  0.004948  0.747253  0.932013
3    3  2022-06-10     50.0  0.988001  0.004924  1.247253  2.179266
4    4  2022-12-10     50.0  0.983232  0.004901  1.747253  3.926519
5    5  2023-06-10  10050.0  0.978486  0.980255  2.247253  6.173771


# Price -> YTM

In [8]:
# 입력값
issuedate = "14/03/10"
maturity = "19/03/10"
settlement = "14/03/10"     # 기간

CR = 0.03125
# YTM = 0.03125   # 금리

period = 6   # 주기

In [9]:
# bond_price = float(input("input price: "))
bond_price = 10893.49

get_irr \
= lambda IRR: calc1.get_price(issuedate, maturity, settlement, CR, IRR, period) - bond_price

estimate = 0

IRR = optimize.newton(get_irr, estimate, disp=False)

[Price]: 11562.5
YTM: 0.0
[Cash Flow]
    num        date        CF   DF  PV of CF
1     1  2014-09-10    156.25  1.0    156.25
2     2  2015-03-10    156.25  1.0    156.25
3     3  2015-09-10    156.25  1.0    156.25
4     4  2016-03-10    156.25  1.0    156.25
5     5  2016-09-10    156.25  1.0    156.25
6     6  2017-03-10    156.25  1.0    156.25
7     7  2017-09-10    156.25  1.0    156.25
8     8  2018-03-10    156.25  1.0    156.25
9     9  2018-09-10    156.25  1.0    156.25
10   10  2019-03-10  10156.25  1.0  10156.25

[Price]: 11557.071773148566
YTM: 0.0001
[Cash Flow]
    num        date        CF       DF      PV of CF
1     1  2014-09-10    156.25  0.99995    156.242188
2     2  2015-03-10    156.25  0.99990    156.234376
3     3  2015-09-10    156.25  0.99985    156.226565
4     4  2016-03-10    156.25  0.99980    156.218754
5     5  2016-09-10    156.25  0.99975    156.210943
6     6  2017-03-10    156.25  0.99970    156.203133
7     7  2017-09-10    156.25  0.99965    1

In [10]:
# 출력
print(f"[input price]: {bond_price}")
print(f"\n[IRR]: {round(IRR, 5)}")

[input price]: 10893.49

[IRR]: 0.01275
